In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F     
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
    

In [7]:
class CNN(nn.Module):
    def __init__(self, input_chnls, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_chnls, out_channels=8, kernel_size=3, padding=1)   
        self.conv2 = nn.Conv2d(8, out_channels=16, kernel_size=3, padding=1)            
        self.conv3 = nn.Conv2d(16, out_channels=32, kernel_size=3, padding=1)           
        self.pool = nn.MaxPool2d(2, 2)

        # After 3 poolings: 200x80 -> 100x40 -> 50x20 -> 25x10
        self.fc1 = nn.Linear(32 * 25 * 10, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [8]:
# class CNN(nn.Module):
#     def __init__(self, input_chnls, num_classes):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(input_chnls, 32, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(32)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.bn2 = nn.BatchNorm2d(64)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.bn3 = nn.BatchNorm2d(128)
#         self.pool = nn.MaxPool2d(2, 2)
        
#         # Adaptive pooling instead of hardcoding dimensions
#         self.gap = nn.AdaptiveAvgPool2d((4, 4))
#         self.fc1 = nn.Linear(128*4*4, 256)
#         self.fc2 = nn.Linear(256, num_classes)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = self.pool(F.relu(self.bn1(self.conv1(x))))
#         x = self.pool(F.relu(self.bn2(self.conv2(x))))
#         x = self.pool(F.relu(self.bn3(self.conv3(x))))
#         x = self.gap(x)
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device

device(type='cuda')

In [11]:
model = CNN(3, 100).to(device)

In [12]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir, transforms = None, class_to_idx = None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir 
        self.transforms = transforms
        self.class_to_idx = class_to_idx



    def __len__(self):
        return len(self.labels_df)
    
    def __getitem__(self, idx):
        img_name = self.labels_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path)
        label = self.labels_df.iloc[idx, 1]
        y_label = self.class_to_idx[label]

        if self.transforms:
            image = self.transforms(image)

        return image, y_label

In [13]:
transform = transforms.Compose([
    transforms.Resize((200, 80)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [14]:
# with open('ai_wordlist.txt', 'r') as f:
#     words = [line.strip() for line in f.readlines()]

# class_to_idx = {word: idx for idx, word in enumerate(words)}

# len(class_to_idx)



In [15]:
import pandas as pd

all_labels = []
for file in [
    "captcha_images/train/labels_easy.csv",
    "captcha_images/train/labels_hard.csv",
    "captcha_images/test/labels_easy.csv",
    "captcha_images/test/labels_hard.csv"
]:
    df = pd.read_csv(file)
    all_labels.extend(df.iloc[:, 1].tolist()) 

unique_labels = sorted(set(all_labels))
class_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

print("Classes found:", len(class_to_idx))
print(list(class_to_idx.items())[:10])  



Classes found: 100
[('adapt', 0), ('agents', 1), ('aiword1', 2), ('aiword10', 3), ('aiword11', 4), ('aiword12', 5), ('aiword13', 6), ('aiword2', 7), ('aiword3', 8), ('aiword4', 9)]


In [16]:
train_dataset_easy = CustomImageDataset(
    csv_file = 'captcha_images/train/labels_easy.csv',
    img_dir = 'captcha_images/train/easy',
    transforms = transform,
    class_to_idx = class_to_idx
)

train_dataset_hard = CustomImageDataset(
    csv_file = 'captcha_images/train/labels_hard.csv',
    img_dir = 'captcha_images/train/hard',
    transforms = transform,
    class_to_idx = class_to_idx
)

train_dataset = ConcatDataset([train_dataset_easy, train_dataset_hard])


test_dataset_easy = CustomImageDataset(
    csv_file = 'captcha_images/test/labels_easy.csv',
    img_dir = 'captcha_images/test/easy',
    transforms = transform,
    class_to_idx = class_to_idx
)

test_dataset_hard = CustomImageDataset(
    csv_file = 'captcha_images/test/labels_hard.csv',
    img_dir = 'captcha_images/test/hard',
    transforms = transform,
    class_to_idx = class_to_idx
)

test_dataset = ConcatDataset([test_dataset_easy, test_dataset_hard])


In [17]:

train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [19]:
criterion = nn.CrossEntropyLoss()

In [20]:
num_epochs = 100

In [21]:
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (data, targets) in enumerate(train_dataloader):
        data = data.to(device = device)
        targets = targets.to(device = device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_dataloader):.4f}')


Epoch [1/100], Loss: 3.8429
Epoch [2/100], Loss: 1.6462
Epoch [3/100], Loss: 0.9061
Epoch [4/100], Loss: 0.6353
Epoch [5/100], Loss: 0.4705
Epoch [6/100], Loss: 0.4074
Epoch [7/100], Loss: 0.3562
Epoch [8/100], Loss: 0.3102
Epoch [9/100], Loss: 0.2951
Epoch [10/100], Loss: 0.2629
Epoch [11/100], Loss: 0.2331
Epoch [12/100], Loss: 0.2120
Epoch [13/100], Loss: 0.2061
Epoch [14/100], Loss: 0.1879
Epoch [15/100], Loss: 0.1761
Epoch [16/100], Loss: 0.1771
Epoch [17/100], Loss: 0.1590
Epoch [18/100], Loss: 0.1522
Epoch [19/100], Loss: 0.1468
Epoch [20/100], Loss: 0.1455
Epoch [21/100], Loss: 0.1348
Epoch [22/100], Loss: 0.1257
Epoch [23/100], Loss: 0.1265
Epoch [24/100], Loss: 0.1206
Epoch [25/100], Loss: 0.1166
Epoch [26/100], Loss: 0.1181
Epoch [27/100], Loss: 0.1123
Epoch [28/100], Loss: 0.1098
Epoch [29/100], Loss: 0.1008
Epoch [30/100], Loss: 0.0938
Epoch [31/100], Loss: 0.1122
Epoch [32/100], Loss: 0.0859
Epoch [33/100], Loss: 0.0954
Epoch [34/100], Loss: 0.1014
Epoch [35/100], Loss: 0

In [23]:
torch.save(model.state_dict(), 'smolCRNN_model.pth')
print("Model saved as CNN_model.pth")

Model saved as CNN_model.pth


In [22]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    acc = float(num_correct) / num_samples
    print(f'Got {num_correct} / {num_samples} with accuracy {acc * 100:.2f}')
    return acc

check_accuracy(train_dataloader, model)
check_accuracy(test_dataloader, model)

Got 9903 / 10000 with accuracy 99.03
Got 2838 / 3000 with accuracy 94.60


0.946

- Number of epochs as 10 - 1% accuracy
- Number of epochs as 100 - 100% accuracy
- Right now model is just memorizing, need to add more samples.

- then i reduced from 6m to 500k parameters and model works fine now.